# US State Food and Demographic Data Acquisition
John Platt

Matt McClements

November 25, 2022

In this notebook, we aim to parse and webscrape current data on obesity scores and fast food popularity. Then, we design a database doing SQL statements that create and populate tables including our data to progress towards answering our main question:

**How is fast food restaurant choice across the US states affected by the number of given fast food restaurants in each state, obesity rates, unemployment rates, and median household income?**

The first part of this notebook will be dedicated to data parsing and web scraping where we read in a total of five different data sets using the following techniques:
* For our first two datasets, we will webscrape them. Three modules will be utilized to obtain html data. With this html data, we will use a function that will obtain the columns and rows needed from the html data to create a dataframe.
* For our last three datasets, we will parse them. We will use a function that reads in a JSON file (represents one of our given datasets in the form of a native structure such as a list of dicionaries) and then creates a pandas dataframe given the data from the JSON file.




# Data Parsing and Web Scraping
In this section, we gather up five data sets in HTML or JSON format using the techniques mentioned above intended to assist in answering the main question.

# Dataset #1 (HTML):
This dataset has current data on consequences, exercise and obesity rankings all combined to create an obesity score for each state (2022). This dataset is obtained in HTML format from the webpage linked to below.

Here is a link to the webpage the table is displayed on-https://wallethub.com/edu/fattest-states/16585

In [ ]:
import requests
from lxml import etree
import io
url='https://wallethub.com/edu/fattest-states/16585'
#Get the html
response=requests.get(url)

#See if the size of the html is greater than 200 lines of code
if response.status_code == 200:
  print('Success')
else:
  print('Fail')

#Print html
response.content

htmlparser=etree.HTMLParser()
#io.BytesIO turns the html string into a file-like object
tree1=etree.parse(io.BytesIO(response.content), htmlparser)
root1=tree1.getroot()
root1
#Access the table using xpath
node_table=root1.xpath('body//table')
for node in node_table[0].iter():
    print(node.text)

Success








None
Overall Rank*


None
State


None
Total Score


None
Obesity & Overweight Prevalence


None
Health Consequences


None
Food & Fitness
None
None
1
West Virginia
75.61
1
1
44
None
2
Mississippi
71.97
2
12
22
None
3
Kentucky
68.96
4
10
21
None
4
Louisiana
68.76
3
26
2
None
5
Arkansas
68.19
6
6
16
None
6
Tennessee
67.70
7
13
4
None
7
Alabama
67.26
9
11
3
None
8
Texas
66.11
5
33
20
None
9
South Carolina
65.81
8
15
25
None
10
Delaware
64.73
12
4
40
None
11
Oklahoma
64.48
10
30
19
None
12
North Carolina
63.26
17
23
1
None
13
Kansas
63.13
19
9
10
None
14
Ohio
62.94
14
21
14
None
15
Iowa
62.31
18
16
29
None
16
Georgia
62.26
15
34
6
None
17
New Mexico
61.88
11
41
45
None
18
Missouri
61.43
16
28
33
None
19
South Dakota
61.41
13
42
8
None
20
Maine
60.86
35
2
23
None
21
Michigan
60.78
24
22
5
None
22
Indiana
60.45
20
29
18
None
23
Virginia
60.36
23
20
26
None
24
North Dakota
60.12
25
19
23
None
25
Maryland
60.08
22
18
35
None
26
Nebraska
59.26
21
38
12
None
27
Nevada
58.76
26
3

Overall, the code above webscrapes the table of obesity scores for each state from the wallethub website utilizing functions of the requests, etree, and io modules. To explain this code a little bit more:
* First, we assign the the url for our website with the needed data to the url variable.
* Then, to get the html corresponding to the url (in the form of a string), we use the get() function of the requests module and assign the result to the variable of response.
* We utilize an if-else statement to determine the html is greater than 200 lines of code using the status_code method.
* We use the functions of etree.HTMLParser() and etree.parse() to parse our html tree assigned to the variable tree1. The io.BytesIO() function as an argument of the etree.parse() function allows us to do this as it converts our html string to a file.
* We get the root of the html tree using tree1.xpath.
* Finally, we use an xpath `('body//table)` to obtain our needed data from the html tree and assign the result to node_table.

Now that we have obtained the table we need through xpath, we can now traverse through the table tree to obtain a list of columns and a list of the row values (LoL). In the code below, we access the columns and row values for our table of obesity scores using a combination of xpath and a list comprehension to obtain a list of lists.





In [ ]:
#Access the columns of the given table
obesity_cols=node_table[0].xpath('./thead/tr/th/p/b/text()')
print(obesity_cols)
#Access the row values
data=node_table[0].xpath('./tbody/tr/td/text()')
print(data)
#Create LoL with each element being a row of our desired table
LoL=[data[i:i+len(obesity_cols)] for i in range(0,len(data),len(obesity_cols))]
LoL

['Overall Rank*', 'State', 'Total Score', 'Obesity & Overweight Prevalence', 'Health Consequences', 'Food & Fitness']
['1', 'West Virginia', '74.60', '1', '1', '9', '2', 'Mississippi', '72.33', '2', '10', '1', '3', 'Kentucky', '68.99', '4', '8', '5', '4', 'Arkansas', '68.95', '3', '17', '6', '5', 'Alabama', '68.63', '5', '11', '3', '6', 'Tennessee', '67.46', '6', '12', '10', '7', 'Louisiana', '65.66', '7', '26', '4', '8', 'Delaware', '63.99', '14', '3', '22', '9', 'Oklahoma', '63.71', '8', '32', '15', '10', 'South Carolina', '63.43', '10', '22', '2', '11', 'Georgia', '62.27', '11', '28', '8', '12', 'Texas', '62.08', '9', '34', '13', '13', 'Ohio', '61.99', '13', '18', '14', '14', 'Missouri', '61.51', '12', '23', '17', '15', 'North Carolina', '60.13', '16', '21', '18', '16', 'Iowa', '60.10', '15', '19', '33', '17', 'Kansas', '59.98', '21', '7', '36', '18', 'Michigan', '58.32', '20', '24', '21', '19', 'Indiana', '58.12', '17', '27', '16', '20', 'Virginia', '57.71', '24', '20', '7', '21', 

[['1', 'West Virginia', '74.60', '1', '1', '9'],
 ['2', 'Mississippi', '72.33', '2', '10', '1'],
 ['3', 'Kentucky', '68.99', '4', '8', '5'],
 ['4', 'Arkansas', '68.95', '3', '17', '6'],
 ['5', 'Alabama', '68.63', '5', '11', '3'],
 ['6', 'Tennessee', '67.46', '6', '12', '10'],
 ['7', 'Louisiana', '65.66', '7', '26', '4'],
 ['8', 'Delaware', '63.99', '14', '3', '22'],
 ['9', 'Oklahoma', '63.71', '8', '32', '15'],
 ['10', 'South Carolina', '63.43', '10', '22', '2'],
 ['11', 'Georgia', '62.27', '11', '28', '8'],
 ['12', 'Texas', '62.08', '9', '34', '13'],
 ['13', 'Ohio', '61.99', '13', '18', '14'],
 ['14', 'Missouri', '61.51', '12', '23', '17'],
 ['15', 'North Carolina', '60.13', '16', '21', '18'],
 ['16', 'Iowa', '60.10', '15', '19', '33'],
 ['17', 'Kansas', '59.98', '21', '7', '36'],
 ['18', 'Michigan', '58.32', '20', '24', '21'],
 ['19', 'Indiana', '58.12', '17', '27', '16'],
 ['20', 'Virginia', '57.71', '24', '20', '7'],
 ['21', 'Maine', '57.53', '34', '2', '45'],
 ['22', 'Rhode Island

* The first xpath above, assigned the variable name obesity_cols, obtains the columns for our first dataset. To explain how this xpath `('./thead/tr/th/p/b/text()')` obtains the columns for our first dataset, the  `.` at the beginning of the path represents the beginning node of the traversal which is the table node. Then, the `/thead` part of the xpath traverses to the thead node of the table which is the table header (child of the table node), the `/tr` part (child of thead) traverses to the first row of the table, and then the `/th` part (children of tr) traverses to the individual header cells of the table where the column names are. Then, the `/p/b` part of the xpath traverses to the p node and then the b node (child of the p node) since both of these nodes come before the text of the column names. Finally, the `/text()` portion of the xpath extracts a list of all the text corresponding to the different b nodes which is where are our desired column names are located in the html tree of the table.

* The second xpath above, assigned the variable name data, obtains the values for all the rows below the columns of our first dataset. This second xpath `('./tbody/tr/td/text())'`, unlike the first xpath above, traverses to the tbody node of the table by doing `./tbody`(child of table node also) which basically involves traversing to everything below the header portion of the table. Then, the `/tr/td` portion of the xpath traverses to all the cells of the table where the needed values are. Finally, the `/text()` portion of the second xpath extracts a list of all the text corresponding to different td nodes which is where are our desired row values are located in the html tree of the table.

* Finally, given our list of columns and list of row values, we create a list of lists using the list comprehension shown above where each element of the list of lists is a subset list of our data list equal to the length of the columns list (since our data list is formatted as `[row1column1value,row1column2value,row1column3value...,row2column1value,row2column2value,row2column3value...]` and the desired list of list is the format of `[[row1column1value,row1column2value,row1column3value...],[row2column1value,row2column2value,row2column3value...]]`). So, stated more simply, each element of our list of lists created by the list comprehension above is the list of all the values below each column for a given row. So, for example, the first element of our list of lists corresponds to all the values below each column at the first row of our dataset.

# pd.DataFrame Function

After we imported the pandas package to recieve the functions, we used the pd.DataFrame function to convert the List of Lists we created above into a pandas dataframe. We did this by using the List of Lists and specifying the columns as the obesity_cols variable that we also created above and we assigned it to the variable 'obesity_df'. Next we used the .head() function to show the first 5 rows of our data set as a way to check our work.


In [ ]:
import pandas as pd

obesity_df = pd.DataFrame(LoL,columns=obesity_cols)
obesity_df.head()

,Overall Rank*,State,Total Score,Obesity & Overweight Prevalence,Health Consequences,Food & Fitness
0,1,West Virginia,74.60,1,1,9
1,2,Mississippi,72.33,2,10,1
2,3,Kentucky,68.99,4,8,5
3,4,Arkansas,68.95,3,17,6
4,5,Alabama,68.63,5,11,3


Below is a description of our variables for the first dataset.

Variable|Variable Description
-|-
Overall Rank*|The overall rank of a given US State based on the overall score.
State|A US State
Total Score|The overall obesity score for a given US State based on obesity prevalence, health consequences, and food & fitness
Overweight & Obesity Prevalence|The rank for a given US State based on Obesity Prevalence (1 is the worst or higher obesity prevalence)
Health Consequences|The rank for a given US State based on Health Consequences (1 is the worst health consequences)
Food & Fitness|The rank for a given US State based on Foot & Fitness (1 is the worst)

# Dataset #2 (HTML):
This dataset has current data on how many fast food restaurants each state has, per 100,000 people (2022). This dataset is obtained in HTML format from the webpage linked to below.

Here is a link to the webpage the table is displayed on-https://www.nicerx.com/fast-food-capitals/



In [ ]:
#Import modules
import requests
from lxml import etree
import io

In [ ]:
url='https://www.nicerx.com/fast-food-capitals/'
#Get the html
response=requests.get(url)

#See if the size of the html is greater than 200 lines of code
if response.status_code == 200:
  print('Success')
else:
  print('Fail')

#Print html
response.content

htmlparser=etree.HTMLParser()
#io.BytesIO converts the html string to a file-like object
tree1=etree.parse(io.BytesIO(response.content), htmlparser)
root1=tree1.getroot()

root1
#Access the table using xpath
node_table=root1.xpath('body/div//table')
for node in node_table[0].iter():
    print(node.text)

Success


None
None
State
All fast food restaurants
Full-service restaurants 
Subway
Starbucks
McDonalds
Dunkin Donut
Burger King
Taco Bell
Dominos
Wendys
Dairy Queen
KFC


None
Hawaii
97.5
89.9
7.0
6.4
5.2
0.8
2.0
2.1
1.6
0.7
0.5
1.1
None
New York
92.3
106.1
4.5
3.3
3.1
7.6
1.8
1.0
1.2
1.1
0.2
0.8
None
Maryland
89.7
59.0
6.8
4.6
4.7
4.7
2.0
1.5
2.0
1.6
0.5
1.1
None
Nevada
84.8
68.4
6.1
8.2
4.6
0.9
2.1
2.6
1.8
1.4
1.2
1.3
None
Ohio
82.8
65.2
8.9
3.8
5.2
1.9
2.9
3.2
2.2
3.5
2.2
1.7
None
Illinois
82.5
77.7
7.8
4.8
5.2
5.5
2.4
2.1
1.6
1.6
2.0
1.2
None
Georgia
82.5
69.4
7.1
3.6
4.2
2.2
2.6
2.3
2.1
2.7
2.0
1.5
None
California
82.3
80.4
5.7
7.4
3.2
0.3
1.4
2.1
1.4
0.7
0.2
1.1
None
Alabama
81.7
57.6
8.2
1.8
5.0
0.9
3.5
2.8
2.5
2.0
1.3
1.8
None
Rhode Island
81.3
110.0
5.9
2.4
2.9
16.4
2.3
1.5
2.5
1.7
0.2
1.3
None
New Jersey
81.2
89.1
2.2
3.3
2.9
10.1
2.1
1.1
1.8
1.6
0.9
0.6
None
Virginia
80.1
81.7
7.3
5.2
4.7
2.8
2.3
2.4
2.6
2.6
1.1
1.4
None
New Hampshire
79.8
100.2
6.3
2.4
4.0
16.2
2.3
1.5
2.

The above code is what we used to get the data (number of fast food restaurants in each state per 100,000 people) from the nicerx website and import it into google colab. To talk about it a little further:
* First we import the URL and assign it to the variable "url". Then we used the .get() function and passed our url variable to import the html file.
* Next we made an if, else statement that tells us if there are more than 200 lines of code in the html. If there is the code prints "success" but if not, the code prints "fail".
* Next, the io.BytesIO() function converts the html string into a file like object and we create the root1 variable to access the root of the html tree.
* Then we created an xpath using the body/div//table nodes to locate the table and a for loop to see the text of that table from the html.
* We end up with the table from the html in a list (list has only one element of our needed table in this case since the website only has one table).







Now that we have the html tree corresponding to the table of our second dataset, we can traverse through that tree to obtain the columns and row values for our second dataset (shown in the code below).

In [ ]:
#Access the columns of the given table
state_ff_cols=node_table[0].xpath('./thead/tr/th/text()')
print(state_ff_cols)
#Access the row values
data=node_table[0].xpath('./tbody/tr/td/text()')
print(data)
#Create LoL with each element being a row of our desired table
LoL=[data[i:i+len(state_ff_cols)] for i in range(0,len(data),len(state_ff_cols))]
LoL

['State', 'All fast food restaurants', 'Full-service restaurants ', 'Subway', 'Starbucks', 'McDonalds', 'Dunkin Donut', 'Burger King', 'Taco Bell', 'Dominos', 'Wendys', 'Dairy Queen', 'KFC']
['Hawaii', '97.5', '89.9', '7.0', '6.4', '5.2', '0.8', '2.0', '2.1', '1.6', '0.7', '0.5', '1.1', 'New York', '92.3', '106.1', '4.5', '3.3', '3.1', '7.6', '1.8', '1.0', '1.2', '1.1', '0.2', '0.8', 'Maryland', '89.7', '59.0', '6.8', '4.6', '4.7', '4.7', '2.0', '1.5', '2.0', '1.6', '0.5', '1.1', 'Nevada', '84.8', '68.4', '6.1', '8.2', '4.6', '0.9', '2.1', '2.6', '1.8', '1.4', '1.2', '1.3', 'Ohio', '82.8', '65.2', '8.9', '3.8', '5.2', '1.9', '2.9', '3.2', '2.2', '3.5', '2.2', '1.7', 'Illinois', '82.5', '77.7', '7.8', '4.8', '5.2', '5.5', '2.4', '2.1', '1.6', '1.6', '2.0', '1.2', 'Georgia', '82.5', '69.4', '7.1', '3.6', '4.2', '2.2', '2.6', '2.3', '2.1', '2.7', '2.0', '1.5', 'California', '82.3', '80.4', '5.7', '7.4', '3.2', '0.3', '1.4', '2.1', '1.4', '0.7', '0.2', '1.1', 'Alabama', '81.7', '57.6', '8.

[['Hawaii',
  '97.5',
  '89.9',
  '7.0',
  '6.4',
  '5.2',
  '0.8',
  '2.0',
  '2.1',
  '1.6',
  '0.7',
  '0.5',
  '1.1'],
 ['New York',
  '92.3',
  '106.1',
  '4.5',
  '3.3',
  '3.1',
  '7.6',
  '1.8',
  '1.0',
  '1.2',
  '1.1',
  '0.2',
  '0.8'],
 ['Maryland',
  '89.7',
  '59.0',
  '6.8',
  '4.6',
  '4.7',
  '4.7',
  '2.0',
  '1.5',
  '2.0',
  '1.6',
  '0.5',
  '1.1'],
 ['Nevada',
  '84.8',
  '68.4',
  '6.1',
  '8.2',
  '4.6',
  '0.9',
  '2.1',
  '2.6',
  '1.8',
  '1.4',
  '1.2',
  '1.3'],
 ['Ohio',
  '82.8',
  '65.2',
  '8.9',
  '3.8',
  '5.2',
  '1.9',
  '2.9',
  '3.2',
  '2.2',
  '3.5',
  '2.2',
  '1.7'],
 ['Illinois',
  '82.5',
  '77.7',
  '7.8',
  '4.8',
  '5.2',
  '5.5',
  '2.4',
  '2.1',
  '1.6',
  '1.6',
  '2.0',
  '1.2'],
 ['Georgia',
  '82.5',
  '69.4',
  '7.1',
  '3.6',
  '4.2',
  '2.2',
  '2.6',
  '2.3',
  '2.1',
  '2.7',
  '2.0',
  '1.5'],
 ['California',
  '82.3',
  '80.4',
  '5.7',
  '7.4',
  '3.2',
  '0.3',
  '1.4',
  '2.1',
  '1.4',
  '0.7',
  '0.2',
  '1.1'],
 ['Ala

* The first xpath above, assigned the variable name state_ff_cols, obtains the columns for our second dataset. To explain how this xpath `('./thead/tr/th/text()')` obtains the columns for our second dataset, the  `.` at the beginning of the path represents the beginning node of the traversal which is the table node. Then, the `/thead` part of the xpath traverses to the thead node of the table which is the table header (child of the table node), the `/tr` part (child of thead) traverses to the first row of the table, and then the `/th` part (children of tr) traverses to the individual header cells of the table where the column names are. Finally, the `/text()` portion of the xpath extracts a list of all the text corresponding to the different th nodes which is where are our desired column names are located in the html tree of the table.

* The second xpath above, assigned the variable name data, obtains the values for all the rows below the columns of our second dataset. This second xpath `('./tbody/tr/td/text())'`, unlike the first xpath above, traverses to the tbody node of the table by doing `./tbody`(child of table node also) which basically involves traversing to everything below the header portion of the table. Then, the `/tr/td` portion of the xpath traverses to all the cells of the table where the needed values are. Finally, the `/text()` portion of the second xpath extracts a list of all the text corresponding to different td nodes which is where are our desired row values are located in the html tree of the table.

* Finally, given our list of columns and list of row values, we create a list of lists using the list comprehension shown above where each element of the list of lists is a subset list of our data list equal to the length of the columns list (since our data list is formatted as `[row1column1value,row1column2value,row1column3value...,row2column1value,row2column2value,row2column3value...]` and the desired list of list is the format of `[[row1column1value,row1column2value,row1column3value...],[row2column1value,row2column2value,row2column3value...]]`). So, stated more simply, each element of our list of lists created by the list comprehension above is the list of all the values below each column for a given row. So, for example, the first element of our list of lists corresponds to all the values below each column at the first row of our dataset.

# pd.DataFrame function

Given the list of lists (LoL) containing our row values and a list of the necessary column names (state_ff_cols), we use the pd.DataFrame function in the code below to finally convert the html representation of our second dataset to a pandas dataframe. We assign the result to state_ff_df and display the first five rows of the resulting dataframe using the head() function.

In [ ]:
import pandas as pd
state_ff_df = pd.DataFrame(LoL,columns=state_ff_cols)
state_ff_df.columns=state_ff_df.columns.str.strip()
state_ff_df

,State,All fast food restaurants,Full-service restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen,KFC
0,Hawaii,97.5,89.9,7.0,6.4,5.2,0.8,2.0,2.1,1.6,0.7,0.5,1.1
1,New York,92.3,106.1,4.5,3.3,3.1,7.6,1.8,1.0,1.2,1.1,0.2,0.8
2,Maryland,89.7,59.0,6.8,4.6,4.7,4.7,2.0,1.5,2.0,1.6,0.5,1.1
3,Nevada,84.8,68.4,6.1,8.2,4.6,0.9,2.1,2.6,1.8,1.4,1.2,1.3
4,Ohio,82.8,65.2,8.9,3.8,5.2,1.9,2.9,3.2,2.2,3.5,2.2,1.7
5,Illinois,82.5,77.7,7.8,4.8,5.2,5.5,2.4,2.1,1.6,1.6,2.0,1.2
6,Georgia,82.5,69.4,7.1,3.6,4.2,2.2,2.6,2.3,2.1,2.7,2.0,1.5
7,California,82.3,80.4,5.7,7.4,3.2,0.3,1.4,2.1,1.4,0.7,0.2,1.1
8,Alabama,81.7,57.6,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,1.8
9,Rhode Island,81.3,110.0,5.9,2.4,2.9,16.4,2.3,1.5,2.5,1.7,0.2,1.3


Below is a description of our variables for the second dataset.

Variable|Variable Description
-|-
State|A US State
All fast food restaurants|The total number of fast food restaurants in each state per 100,000 people
Full-service restaurants|The total number of full service restaurants in each state per 100,000 people
Subway|The total number of Subways in each state per 100,000 people
Starbucks|The total number of Starbucks in each state per 100,000 people
McDonalds|The total number of McDonalds in each state per 100,000 people
Dunkin Donut|The total number of Dunkin Donuts in each state per 100,000 people
Burger King|The total number of Burger Kings in each state per 100,000 people
Taco Bell|The total number of Taco Bells in each state per 100,000 people
Dominos|The total number of Dominos in each state per 100,000 people
Wendys|The total number of Wendys in each state per 100,000 people
Dairy Queen|The total number of Dairy Queens in each state per 100,000 people
KFC|The total number of KFCs in each state per 100,000 people

# Our JSON Conversion Function

This funtion, given a JSON file name, first uses the open() and the JSON.load() functions to open a JSON file and read in the contents of a JSON file downloaded from our data (list of dictionaries (LOD) in our case) respectively. Then the function has the return statement of the pd.DataFrame function to convert the LOD obtained from the JSON data into a pandas dataframe. This function will be used to convert our next three datasets represented as JSON files to pandas dataframes.

In [ ]:
def convertLOD(file):
  import json
  import pandas as pd

  with open(file, 'r') as fi:
    DOL = json.load(fi)

  return  pd.DataFrame(DOL)




# Dataset #3 (JSON):
This dataset consists of data on current median household income for each US state (2022). This dataset is obtained in JSON format.

Here is a link to the webpage the table is displayed on-https://worldpopulationreview.com/state-rankings/median-household-income-by-state

Here is a link to download the **needed** JSON file that represents this dataset as a DoL-https://drive.google.com/file/d/1mlu8XnDPcNrR098xhII3VICPj8MnT4mg/view?usp=sharing


In [ ]:
medIncome=convertLOD('data.json')
medIncome.columns=medIncome.columns.str.replace('state','State')
medIncome

NameError: name 'convertLOD' is not defined

The code above is taking in the JSON file about the median household income in each US state in 2022, and converting that data into a pandas dataframe. It does this by calling the convertDOL() function that we created above and using the downloaded json file for the data set as the parameter.

Below is a description of each of our variables of the third dataset.

Variable|Variable Description
-|-
State|A US State
HouseholdIncome|The Median income of every household in each respective state

# Dataset #4 (JSON):
This dataset contains data on unemployment rate by US state (2021 and 2022). This dataset is obtained in JSON format.

Here is a link to the webpage the table is displayed on-https://worldpopulationreview.com/state-rankings/unemployment-rate-by-state

Here is a link to download the **needed** JSON file that represents this dataset as a DoL- https://drive.google.com/file/d/1HIQZqRWuxNPAn4uhDYc7VDLry-8kfx6_/view?usp=sharing

In [ ]:
unemployment=convertLOD('data (1).json')
unemployment.columns=unemployment.columns.str.replace('state','State')
unemployment

,State,rate22,rate21
0,Minnesota,1.8000,3.3000
1,Nebraska,2,2.5000
2,New Hampshire,2,3.5000
3,Utah,2,2.7000
4,Vermont,2.1000,3.3000
5,North Dakota,2.3000,3.4000
6,South Dakota,2.3000,3.2000
7,Kansas,2.4000,3.4000
8,Iowa,2.5000,4.4000
9,Missouri,2.5000,4.3000


The code above is taking in the JSON file about unemployment rates in the US in 2021 and 2022, and converting that data into a pandas dataframe. It does this by calling the convertDOL() function that we created and using the downloaded json file for the data set as the parameter.

Below is a description of each of the variables in our fourth dataset.

Variable|Variable Description
-|-
State|A US State
rate22|A given state's unemployment rate as of July 2022
rate21|A given state's unemployment rate as of July 2021

# Dataset #5 (JSON):
This dataset contains current data on the most popular fast food restaurants in each US state (2022). This dataset is obtained in JSON format.

Here is a link to the webpage the table is displayed on-https://worldpopulationreview.com/state-rankings/most-popular-fast-food-by-state

Here is a link to download the **needed** JSON file that represents this dataset as a DoL-https://drive.google.com/file/d/1XUgT8uEcHKLPtDzC0V_QJULDPEnMCe7-/view?usp=sharing


In [ ]:
fastfoodfav=convertLOD('data (2).json')
fastfoodfav.columns=fastfoodfav.columns.str.replace('state','State')
fastfoodfav

,State,mostPopular
0,Illinois,White Castle
1,Iowa,Wendy's
2,Vermont,Taco Bell
3,Montana,Sonic
4,Connecticut,Panda Express
5,Hawaii,Panda Express
6,Minnesota,Panda Express
7,New Jersey,Panda Express
8,Oregon,Panda Express
9,Alaska,McDonald's


The code above is taking in the JSON file about the most popular fast food resurants in the US in 2022, and converting that data into a pandas dataframe. It does this by calling the convertDOL() function that we created  and using the downloaded json file for the data set as the parameter.

Below is a description of each of our variables of our fifth dataset.

Variable|Variable Description
-|-
State|A US State
mostPopular|The most popular fast food resturant in each respective state

**Our plan to use the data to answer our main question:**

We intend to use the variables of the obesity score, median household income, and unemployment rate of our obesity data (dataset #1 above), median household income data (dataset #3 above), and unemployment data (dataset #4) respectively. These variables will serve as our explanatory variables that we think may help to explain fast food popularity. Then, the variable of the number of certain fast food restaurants per 100,000 people for each state (Dataset #2) will be utilized as another explanatory variable in our analysis. Finally, variable of the favorite fast food restaurant in each state from dataset #5 will be our response variable that we think is explained by the variables above. The second half of this notebook will involve creating a database from our given data using SQL statements to create and populate tables. Then another notebook will be created that will perform analysis given different dataframes obtained from our database.

The analysis in the second notebook will include the following methods:
* Describing our data using comparative boxplots, histograms, barplots, or any other appropriate method.

* Performing correlation analysis on our data creating scatterplots, computing correlation coefficients, or any other appropriate method.


# Database Design
In this section, we design a database using SQL statements to create and populate tables. Yet, first, we need to do a little bit a data manipulation before we create any tables our database that follow proper database design. We do data manipulation below.

# Using the Melt() Function to rearrange the obesity data set and make it tidy

This code we used the melt fuction to re-arrange oue obesity data and make it much more tidy by putting our category values spanned over multiple columns (Health Consequences, Obesity & Overweight Prevalence...) below one column called Category and the rankings for those different categories below another column called ranking. After adding a unique identifier column of categoryID for when we put a table like this in our database and deleteing any rows that contain the District of Columbia, we end up with a data frame with all of the states, with their rankings and obesity scores based on their categories. We plan to use this data in our data base to eventually make plots of how this data distributes.

In [ ]:
obesity_df1 = obesity_df.melt(id_vars = ['State','Overall Rank*','Total Score'],
                             var_name='Category', value_name = 'Ranking')
obesity_df1.columns=obesity_df1.columns.str.replace('*','').str.strip()
obesity_df1.sort_values(by=['Category','State'],inplace=True)
#Delete any rows with District of Columbia
obesity_df1=obesity_df1[obesity_df1['State']!='District of Columbia']
#Add catID column to unique identify each row
obesityIDs=[i for i in range(101,251)]
obesity_df1.insert(0,'categoryID',obesityIDs)
obesity_df1

<ipython-input-12-ca1c1c6e4605>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  obesity_df1.columns=obesity_df1.columns.str.replace('*','').str.strip()


,categoryID,State,Overall Rank,Total Score,Category,Ranking
106,101,Alabama,5,68.63,Food & Fitness,3
135,102,Alaska,34,53.84,Food & Fitness,46
144,103,Arizona,43,51.12,Food & Fitness,25
105,104,Arkansas,4,68.95,Food & Fitness,6
146,105,California,45,49.97,Food & Fitness,26
...,...,...,...,...,...,...
19,246,Virginia,20,57.71,Obesity & Overweight Prevalence,24
41,247,Washington,42,51.17,Obesity & Overweight Prevalence,37
0,248,West Virginia,1,74.60,Obesity & Overweight Prevalence,1
27,249,Wisconsin,28,55.51,Obesity & Overweight Prevalence,29


# Obtain obesity_by_state dataset
Here, we obtain a dataset called obesity_by_state that will include the overall score and overall rank for each state in the US. We intend to create a dataset very similar to the obesity_by_state dataframe displayed below when we create our database.

In [ ]:
#Get rid district of columbia
obesity_df=obesity_df[obesity_df['State']!='District of Columbia']
obesity_by_state=obesity_df[['State','Overall Rank*','Total Score']]
obesity_by_state=obesity_by_state.sort_values(by='State')
#Add unique identifier column
stateID=[i for i in range(101,151)]
obesity_by_state.insert(0,'StateID',stateID)
obesity_by_state

,StateID,State,Overall Rank*,Total Score
4,101,Alabama,5,68.63
33,102,Alaska,34,53.84
42,103,Arizona,43,51.12
3,104,Arkansas,4,68.95
44,105,California,45,49.97
50,106,Colorado,51,43.23
37,107,Connecticut,38,52.69
7,108,Delaware,8,63.99
30,109,Florida,31,54.63
10,110,Georgia,11,62.27


# Obtain category_by_state dataset

The category_by_state data frame comes directly from our obesity_df1 data frame but only uses the "categoryID, State, Category, Ranking" variables. Later in this notebook, ew will take this data and put it directly into our final database to research trends in states and which rankings states fall under for each category.

In [ ]:
category_by_state=obesity_df1[['categoryID','State','Category','Ranking']]
category_by_state

,categoryID,State,Category,Ranking
106,101,Alabama,Food & Fitness,3
135,102,Alaska,Food & Fitness,46
144,103,Arizona,Food & Fitness,25
105,104,Arkansas,Food & Fitness,6
146,105,California,Food & Fitness,26
...,...,...,...,...
19,246,Virginia,Obesity & Overweight Prevalence,24
41,247,Washington,Obesity & Overweight Prevalence,37
0,248,West Virginia,Obesity & Overweight Prevalence,1
27,249,Wisconsin,Obesity & Overweight Prevalence,29


# Combining the fast food data
Here, we combine our dataset about the number of different fast food restaurants per 100,000 people (dataset #2) with our dataset about the most popular fast food restaurants (dataset #5). We will utilize subsets of data from this table to create one table that will be in our database called state_fast_food (will have information about the favorite fast food restaurants in each state and other columns from the table below) and another called state_numfastfood that will be in our database (will have information about the number of different fast food restaurants per 100,000 people for each of our states).

In [ ]:
import pandas as pd
state_ff_df1=state_ff_df.copy()
fastfoodfav1=fastfoodfav.copy()
#Set the index as State for both tables
state_ff_df1=state_ff_df1.set_index('State')
fastfoodfav1=fastfoodfav1.set_index('State')
#Combine the fast food data hoping the rows and so the states are identical
state_ff=pd.concat([state_ff_df1,fastfoodfav1],axis=1,sort=True)
#Make the States a column of the state_ff dataframe again
state_ff['State']=state_ff.index
state_ff

,All fast food restaurants,Full-service restaurants,Subway,Starbucks,McDonalds,Dunkin Donut,Burger King,Taco Bell,Dominos,Wendys,Dairy Queen,KFC,mostPopular,State
State,,,,,,,,,,,,,,
Alabama,81.7,57.6,8.2,1.8,5.0,0.9,3.5,2.8,2.5,2.0,1.3,1.8,Chick-Fil-A,Alabama
Alaska,61.9,78.7,7.2,6.6,4.2,0.3,1.1,2.1,1.4,1.2,0.7,0.8,McDonald's,Alaska
Arizona,67.9,59.2,5.6,7.3,3.9,1.3,2.5,2.5,1.6,1.4,1.5,0.8,In-N-Out,Arizona
Arkansas,69.9,67.6,8.4,2.0,5.7,0.3,2.7,3.5,2.2,2.1,1.0,2.0,Chick-Fil-A,Arkansas
California,82.3,80.4,5.7,7.4,3.2,0.3,1.4,2.1,1.4,0.7,0.2,1.1,In-N-Out,California
Colorado,75.7,88.2,6.7,8.8,3.6,0.8,2.0,2.6,2.3,2.3,1.5,1.2,Chick-Fil-A,Colorado
Connecticut,76.1,97.1,8.1,3.4,4.0,14.4,1.8,1.4,1.5,1.5,1.1,1.0,Panda Express,Connecticut
Delaware,78.3,85.5,2.5,3.6,3.7,7.1,2.1,1.5,2.7,1.2,1.5,1.1,Chick-Fil-A,Delaware
Florida,65.5,78.9,6.0,3.6,4.2,4.1,2.7,2.1,1.9,2.5,0.7,1.2,Chick-Fil-A,Florida


# Obtain state_fast_food dataset


In this cell we created our state_fast_food data frame. We first took our state_ff data and chose which columns to put in the new data drame. Then we created a unique identifier column (StateID) so we could have a unique identifier column to have a primary key when we put a table similar to this table in our database. We intend to use this data to populate a table named state_fast_food in our final product database.

In [ ]:
state_fast_food=state_ff[['State','All fast food restaurants','Full-service restaurants','mostPopular']]
#Create unique identifier column
stateID=[i for i in range(101,151)]
state_fast_food.insert(0,'StateID',stateID)
state_fast_food

,StateID,State,All fast food restaurants,Full-service restaurants,mostPopular
State,,,,,
Alabama,101,Alabama,81.7,57.6,Chick-Fil-A
Alaska,102,Alaska,61.9,78.7,McDonald's
Arizona,103,Arizona,67.9,59.2,In-N-Out
Arkansas,104,Arkansas,69.9,67.6,Chick-Fil-A
California,105,California,82.3,80.4,In-N-Out
Colorado,106,Colorado,75.7,88.2,Chick-Fil-A
Connecticut,107,Connecticut,76.1,97.1,Panda Express
Delaware,108,Delaware,78.3,85.5,Chick-Fil-A
Florida,109,Florida,65.5,78.9,Chick-Fil-A


# Using the melt() function to rearrange the fastfood dataset and make it tidy
Here, we use the melt() function to put the values that are spanned over multiple columns from our state_ff dataset (Subway,KFC,McDonalds,Starbucks...) below one one column called "Fast Food Restaurant" and numerical records corresponding to each of those fast food restaurants below another column called "Number Per 100 People". We also add a FastFoodID column because we will need to have values for a unique identifier column from this table for our database. We name the resulting dataframe state_ff1. We intend to extract the Fast Food Restaurant, Number Per 100 people, and state/ID columns of this table to create another table called state_numfastfood that will be in our database.

In [ ]:
state_ff1=state_ff.copy()
state_ff1 = state_ff1.melt(id_vars = ['State','All fast food restaurants','Full-service restaurants','mostPopular'],
                              var_name='Fast Food Resturant', value_name = 'Number Per 100 people')
#Add FastFoodID column to unique identify each row
fastFoodIDs=[i for i in range(101,601)]
state_ff1.insert(0,'FastFoodID',fastFoodIDs)
state_ff1

,FastFoodID,State,All fast food restaurants,Full-service restaurants,mostPopular,Fast Food Resturant,Number Per 100 people
0,101,Alabama,81.7,57.6,Chick-Fil-A,Subway,8.2
1,102,Alaska,61.9,78.7,McDonald's,Subway,7.2
2,103,Arizona,67.9,59.2,In-N-Out,Subway,5.6
3,104,Arkansas,69.9,67.6,Chick-Fil-A,Subway,8.4
4,105,California,82.3,80.4,In-N-Out,Subway,5.7
...,...,...,...,...,...,...,...
495,596,Virginia,80.1,81.7,Chick-Fil-A,KFC,1.4
496,597,Washington,66.8,85.1,Charley's,KFC,1.0
497,598,West Virginia,73.8,66.2,McDonald's,KFC,2.1
498,599,Wisconsin,65.0,86.4,McDonald's,KFC,1.1


# Obtain state_numfastfood dataset


This cell creates our state_numfastfood data set that we eventually use values for to create a table with the same name of state_numfastfood in our final database.

In [ ]:
state_numfastfood=state_ff1[['FastFoodID','State','Fast Food Resturant','Number Per 100 people']]
state_numfastfood

,FastFoodID,State,Fast Food Resturant,Number Per 100 people
0,101,Alabama,Subway,8.2
1,102,Alaska,Subway,7.2
2,103,Arizona,Subway,5.6
3,104,Arkansas,Subway,8.4
4,105,California,Subway,5.7
...,...,...,...,...
495,596,Virginia,KFC,1.4
496,597,Washington,KFC,1.0
497,598,West Virginia,KFC,2.1
498,599,Wisconsin,KFC,1.1


# Combining our socioeconomic data
Here we combine the data we have about unemployment rate for different years and median income for each US state. We intend to extract the State and HouseholdIncome columns from this table to create another table called state_medincome that will be in our database.

In [ ]:
import pandas as pd
medIncome1=medIncome.copy()
unemployment1=unemployment.copy()
#Set the index as State for both tables
medIncome1=medIncome1.set_index('State')
unemployment1=unemployment1.set_index('State')
#Combine the socioeconomic data hoping the rows and so the states are identical
socioeconomic=pd.concat([medIncome1,unemployment1],axis=1,sort=True)
#Make the States a column of the state_ff dataframe again
socioeconomic['State']=socioeconomic.index
socioeconomic

,HouseholdIncome,rate22,rate21,State
State,,,,
Alabama,52035,2.6000,3.4000,Alabama
Alaska,77790,4.5000,6.5000,Alaska
Arizona,61529,3.3000,4.9000,Arizona
Arkansas,49475,3.3000,4,Arkansas
California,78672,3.9000,7.4000,California
Colorado,75231,3.3000,5.6000,Colorado
Connecticut,79855,3.7000,6.3000,Connecticut
Delaware,69110,4.4000,5.5000,Delaware
Florida,57703,2.7000,4.5000,Florida


# Obtaining state_medincome dataset

Above we used the 'State' and 'HouseholdIncome' columns from the socioeconomic data frame to create the state_medincome data frame. We then added a unique identifier so we could have a primary key and we plan to use this data directly in our final database to see median income trands among states.

In [ ]:
state_medincome=socioeconomic[['State','HouseholdIncome']]
#Create unique identifier column
stateID=[i for i in range(101,151)]
state_medincome.insert(0,'StateID',stateID)
state_medincome

,StateID,State,HouseholdIncome
State,,,
Alabama,101,Alabama,52035
Alaska,102,Alaska,77790
Arizona,103,Arizona,61529
Arkansas,104,Arkansas,49475
California,105,California,78672
Colorado,106,Colorado,75231
Connecticut,107,Connecticut,79855
Delaware,108,Delaware,69110
Florida,109,Florida,57703


# Using the melt() function to rearrange the socioeconomic dataset and make it tidy
Here, we use the melt() function to put the values that are spanned over multiple columns from our socioeconomic dataset (rate21,rate22) below one one column called "Unemp_Year" and numerical records corresponding to each of those years below another column called "Unemployment_rate". We also add a soceconID column because we will need to have values for a unique identifier column from this table for our database. We name the resulting dataframe socioeconomic1. We intend to extract the Unemp_Year, Unemployment_rate, and state/ID columns of this table to create another table called state_unemp that will be in our database.

In [ ]:
socioeconomic1=socioeconomic.copy()
socioeconomic1=socioeconomic1.melt(id_vars=['State','HouseholdIncome'], var_name='Unemp_Year', value_name='Unemployment_rate')
#Format the years as 2020 and 2021 with their datatype being integers
socioeconomic1['Unemp_Year']=socioeconomic1.Unemp_Year.str.replace('rate','20')
socioeconomic1['Unemp_Year']=socioeconomic1['Unemp_Year'].astype(int)
#Rename state column
socioeconomic1.columns=socioeconomic1.columns.str.replace('state','State')
#Rearrange columns to where the independent variables are first
socioeconomic1=socioeconomic1[['State','Unemp_Year','HouseholdIncome','Unemployment_rate']]
#Add a soceconID column to uniquely identify each row
soceconIDs=[i for i in range(101,201)]
socioeconomic1.insert(0,'soceconID',soceconIDs)
socioeconomic1['Unemployment_rate']=socioeconomic1['Unemployment_rate'].astype(float)
socioeconomic1

,soceconID,State,Unemp_Year,HouseholdIncome,Unemployment_rate
0,101,Alabama,2022,52035,2.6
1,102,Alaska,2022,77790,4.5
2,103,Arizona,2022,61529,3.3
3,104,Arkansas,2022,49475,3.3
4,105,California,2022,78672,3.9
...,...,...,...,...,...
95,196,Virginia,2021,76398,3.8
96,197,Washington,2021,77006,5.2
97,198,West Virginia,2021,48037,5.1
98,199,Wisconsin,2021,63293,3.8


# Obtaining state_unemp dataset


Here we created the state_unemp data set from the previously melted and tidy socioeconomic1. This data will go directly in our final database to see any trends or changes between unemployment rates from 2021 - 2022.

In [ ]:
state_unemp=socioeconomic1[['soceconID','State','Unemp_Year','Unemployment_rate']]
state_unemp

,soceconID,State,Unemp_Year,Unemployment_rate
0,101,Alabama,2022,2.6
1,102,Alaska,2022,4.5
2,103,Arizona,2022,3.3
3,104,Arkansas,2022,3.3
4,105,California,2022,3.9
...,...,...,...,...
95,196,Virginia,2021,3.8
96,197,Washington,2021,5.2
97,198,West Virginia,2021,5.1
98,199,Wisconsin,2021,3.8


# Creating our database file and our tables
We will call our database file US_State_Food.db

# sql_create(db,qry) function

The sql_create function takes in a data base and a query to create a table inside of our data base that contains columns but not yet data. We also don't need a return statement because we are creating the tables down below instead trying to extract data with the fetchall function. We plan to use this function to create all of the tables in our database file.

In [ ]:
def sql_create(db, qry):
  import sqlite3 as sql
  connection=sql.connect(db)
  cur = connection.cursor()
  cur.execute(qry)

# Creating our state obesity and state category tables in our database
Here, we create two different tables (not populated yet) that will be in our database called obesity_by_state and category_by_state. We do this through passing our database name and the queries below in the sql_create function to create the needed tables. Since no values appear to be missing in the data we have extracted in this notebook, we put "NOT NULL" next to all the field names in the query statements that create the two tables below. The state and category columns for our data above both have all string values. Thus, we have the datatype of both columns be VARCHAR(n) where n is the maximum length we want a string below a column to be. Then, all the other columns in both tables below will have data types of INT since those are the intended datatypes for our total_score, overall_rank, ranking, StateID, and CategoryID columns in this notebook.

In [ ]:
db = 'US_State_Food.db'
cqry = '''
CREATE TABLE obesity_by_state (
  StateID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  Overall_Rank INT NOT NULL,
  Total_Score INT NOT NULL,
  PRIMARY KEY (StateID)
  )
'''
sql_create(db,cqry)

The obesity_by_state table above will show the total obesity score and overall rank across each state in the US with the primary key of StateID which will be our unique identifier column.

In [ ]:
db = 'US_State_Food.db'
cqry = '''
CREATE TABLE category_by_state (
  CategoryID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  Category VARCHAR(50) NOT NULL,
  Ranking INT NOT NULL,
  PRIMARY KEY (CategoryID)
  )
'''
sql_create(db,cqry)

The categeory_by_state table above will show the ranking for each category across each state in the US with the primary key of CategoryID which will be our unique identifier column.

# Creating our state median income and state unemployment tables in our database
Here, we create two different tables (not populated yet) that will be in our database called state_medincome and state_unemp. We do this through passing our database name and the queries below in the sql_create function to create the needed tables. We say "NOT NULL" for all the columns we create in the tables below since we don't seem to have any missing values for the data in the tables we have in this notebook. The state column for our data above has string values. Thus, we have the datatype of the state column be VARCHAR(n) where n is the maximum length we want a string below the column to be. We then have the data type of our Unemployment_rate column below be FLOAT since it typically is not a whole number. Then, all the other columns in both tables below will have data types of INT since those are the intended datatypes for our HouseholdIncome, unempID, StateID, and Unemp_Year columns in this notebook.

In [ ]:
db = 'US_State_Food.db'
gqry = '''
CREATE TABLE state_medincome (
  StateID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  HouseholdIncome INT NOT NULL,
  PRIMARY KEY (StateID)
  )
'''
sql_create(db,gqry)

We are creating a table called state_medincome with State as our column of strings of maximum length 25 (VARCHAR(25)) and all of the other columns having numeric/INT data types. We use StateID as our primary key since that will be our unique identifier column in this case. These columns will be the ones we will populate with values later in this notebook in the state_medincome table of our final database file.


In [ ]:
db = 'US_State_Food.db'
gqry = '''
CREATE TABLE state_unemp (
  unempID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  Unemp_Year INT NOT NULL,
  Unemployment_rate FLOAT DEFAULT 0.0,
  PRIMARY KEY (unempID)
  )
'''
sql_create(db,gqry)

We are creating a table called state_unemp with State as our column of strings of maximum length 25 (VARCHAR(25)), the Unemployment_rate being a column of float numbers (FLOAT), and all of the other columns having numeric/INT data types. We use to unempID column as our primary key since that will be our unique identifier column for this table. These columns will be the ones we will populate with values later in this notebook in the state_unemp table of our final database file.

# Creating our state fast food tables in our database
Here, we create two different tables (not populated yet) that will be in our database called state_numfastfood and state_fast_food. We do this through passing our database name and the queries below in the sql_create function to create the needed tables. Since no values appear to be missing in the data we have extracted in this notebook, we put "NOT NULL" next to all the field names in the query statements that create the two tables below. The state, mostPopular, and Fast_Food_Restaurant columns for our data above all have string values. Thus, we have the datatype of the columns be VARCHAR(n) where n is the maximum length we want a string below a column to be. Then, all the other columns in both tables below will have data types of INT since those are the datatypes for our total_score, overall_rank, ranking, StateID, and CategoryID data in this notebook.

In [ ]:
db = 'US_State_Food.db'
bqry = '''
CREATE TABLE state_numfastfood (
  FastFoodID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  Fast_Food_Resturant VARCHAR(25) NOT NULL,
  Number_Per_100_people INT NOT NULL,
  PRIMARY KEY (FastFoodID)
  )
'''
sql_create(db,bqry)

The state_numfastfood table created above will show the number of different fast food restaurants per 100,000 people across each state in the US with the primary key of FastFoodID which will be our unique identifier column.

In [ ]:
db = 'US_State_Food.db'
bqry = '''
CREATE TABLE state_fast_food (
  StateID INT NOT NULL,
  State VARCHAR(25) NOT NULL,
  All_fast_food_restaurants INT NOT NULL,
  Full_service_restaurants INT NOT NULL,
  mostPopular VARCHAR(25) NOT NULL,
  PRIMARY KEY (StateID)
  )
'''
sql_create(db,bqry)

The state_fast_food table created above will show the favorite fast food restaurants, total number of fast food restaurants per 100,000 people, and total number of total service restaurants per 100,000 people across each state in the US with the primary key of StateID which will be our unique identifier column.

# execute_sql(db,sql_q) function

The execute_sql function takes in our data base file and the created sql to make sure that the data that we entered into our tables actually runs correctly. This is important to or main data base because we can be 100% certain that our data is correctly inputed.

In [ ]:
def execute_sql(db, sql_q):
  import sqlite3 as sql
  conn = sql.connect(db)
  cur = conn.cursor()

  result = cur.execute(sql_q)

  return result.fetchall()

# Using the execute_sql function to check that all our tables are in the database

Here, after using the query statement below that allows us to see the names of all the tables in our database, we confirm that our query statements to create the six tables above worked since we have the correct tables of obesity_by_state, category_by_state, state_medincome, state_unemp, state_fast_food, and state_numfastfood in our database.

In [ ]:
sqry='SELECT name FROM sqlite_master'
execute_sql(db,sqry)

[('obesity_by_state',),
 ('sqlite_autoindex_obesity_by_state_1',),
 ('category_by_state',),
 ('sqlite_autoindex_category_by_state_1',),
 ('state_medincome',),
 ('sqlite_autoindex_state_medincome_1',),
 ('state_unemp',),
 ('sqlite_autoindex_state_unemp_1',),
 ('state_numfastfood',),
 ('sqlite_autoindex_state_numfastfood_1',),
 ('state_fast_food',),
 ('sqlite_autoindex_state_fast_food_1',)]

# Inserting values into our tables

# sql_insert_many(db, qry, data) function

For this function called sql_insert_many that takes in a data base, query and a list of tuples, we use it to input the data from just a LoT (list of tuples) into a table of our database. The executemany function below is very important to allow all the value inserts from a list of tuples to occur. This function will be very helpful to our final database because it lets us insert data into all the tables of our database.

In [ ]:
def sql_insert_many(db, qry, data):
  import sqlite3 as sql
  conn = sql.connect(db)
  cur = conn.cursor()
  cur.executemany(qry, data)
  conn.commit()

# Inserting values into the obesity and category tables in our database

# Obtaining a list of tuples from our obesity dataframe in this notebook
Here, we call all the columns in our obesity_by_state dataframe created in this notebook inside the zip function to create an object that has individual tuples with the element of each tuple corresponding to a given value below one of the columns of our obesity_by_state dataframe (e.g. the element at index 0 of each tuple corresponds to the StateID column, the element at index 1 of each tuple corresponds to the State column...). We then call this zip function with its arguments inside the list function to obtain the needed list of tuples from our obesity_by_state data in this notebook. This list of tuples will be used to populate the obesity_by_state table in our database.

In [ ]:
#Obtain list of tuples given our tidy obesity table (obesity_by_state) in this notebook
LoT=list(zip(obesity_by_state['StateID'],obesity_by_state['State'],obesity_by_state['Overall Rank*'],obesity_by_state['Total Score']))
LoT

[(101, 'Alabama', '5', '68.63'),
 (102, 'Alaska', '34', '53.84'),
 (103, 'Arizona', '43', '51.12'),
 (104, 'Arkansas', '4', '68.95'),
 (105, 'California', '45', '49.97'),
 (106, 'Colorado', '51', '43.23'),
 (107, 'Connecticut', '38', '52.69'),
 (108, 'Delaware', '8', '63.99'),
 (109, 'Florida', '31', '54.63'),
 (110, 'Georgia', '11', '62.27'),
 (111, 'Hawaii', '47', '47.00'),
 (112, 'Idaho', '37', '52.71'),
 (113, 'Illinois', '23', '56.73'),
 (114, 'Indiana', '19', '58.12'),
 (115, 'Iowa', '16', '60.10'),
 (116, 'Kansas', '17', '59.98'),
 (117, 'Kentucky', '3', '68.99'),
 (118, 'Louisiana', '7', '65.66'),
 (119, 'Maine', '21', '57.53'),
 (120, 'Maryland', '25', '56.47'),
 (121, 'Massachusetts', '48', '45.90'),
 (122, 'Michigan', '18', '58.32'),
 (123, 'Minnesota', '46', '48.27'),
 (124, 'Mississippi', '2', '72.33'),
 (125, 'Missouri', '14', '61.51'),
 (126, 'Montana', '36', '52.71'),
 (127, 'Nebraska', '41', '51.91'),
 (128, 'Nevada', '32', '54.01'),
 (129, 'New Hampshire', '29', '55.3

# Inserting values into obesity_by_state given list of tuples

This cell populates the obesity_by_state table of our final database with values. We use our sql_insert_many function to take the LoT above to populate the database table of obesity_by_state and end up with a final dataframe called obesity_by_state with its values in our database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 4 total fields in the obesity table of our database, we have 4 question marks in this iqry
iqry = 'INSERT INTO obesity_by_state VALUES(?,?,?,?)'
#Insert values into the obesity table of our database
sql_insert_many(db,iqry,LoT)

# Using the execute_sql function to check that our insert worked

Here we call our execute_sql function with our database and another argument to access all the columns of the obesity_by_state table in our database. From there, we create a dataframe from this query execution to determine if our insert worked and it appears that the insert did work.


In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM obesity_by_state')
cols='StateID,State,Overall_Rank,Total_Score'
obesity_by_state=pd.DataFrame(data,columns=cols.split(','))
obesity_by_state

,StateID,State,Overall_Rank,Total_Score
0,101,Alabama,5,68.63
1,102,Alaska,34,53.84
2,103,Arizona,43,51.12
3,104,Arkansas,4,68.95
4,105,California,45,49.97
5,106,Colorado,51,43.23
6,107,Connecticut,38,52.69
7,108,Delaware,8,63.99
8,109,Florida,31,54.63
9,110,Georgia,11,62.27


# Obtaining list of tuples from our category dataframe in this notebook

Here we take our category_by_state table and create a list of tuples from it. The list of tuples is created by calling all the columns in our category_by_state dataframe created in this notebook inside the zip function to create an object that has individual tuples with the element of each tuple corresponding to a given value below one of the columns of our category_by_state dataframe (e.g. the element at index 0 of each tuple corresponds to the categoryID column, the element at index 1 of each tuple corresponds to the State column...). We then call this zip function with its arguments inside the list function to obtain the needed list of tuples from our category_by_state dataframe in this notebook. We plan on next taking this LoT and using our sql_insert_many function to populate the category dataframe in our database with values.

In [ ]:
#Obtain list of tuples given our tidy category table (category_by_state) in this notebook
LoT=list(zip(category_by_state['categoryID'],category_by_state['State'],category_by_state['Category'],category_by_state['Ranking']))
LoT

[(101, 'Alabama', 'Food & Fitness', '3'),
 (102, 'Alaska', 'Food & Fitness', '46'),
 (103, 'Arizona', 'Food & Fitness', '25'),
 (104, 'Arkansas', 'Food & Fitness', '6'),
 (105, 'California', 'Food & Fitness', '26'),
 (106, 'Colorado', 'Food & Fitness', '19'),
 (107, 'Connecticut', 'Food & Fitness', '38'),
 (108, 'Delaware', 'Food & Fitness', '22'),
 (109, 'Florida', 'Food & Fitness', '11'),
 (110, 'Georgia', 'Food & Fitness', '8'),
 (111, 'Hawaii', 'Food & Fitness', '41'),
 (112, 'Idaho', 'Food & Fitness', '44'),
 (113, 'Illinois', 'Food & Fitness', '12'),
 (114, 'Indiana', 'Food & Fitness', '16'),
 (115, 'Iowa', 'Food & Fitness', '33'),
 (116, 'Kansas', 'Food & Fitness', '36'),
 (117, 'Kentucky', 'Food & Fitness', '5'),
 (118, 'Louisiana', 'Food & Fitness', '4'),
 (119, 'Maine', 'Food & Fitness', '45'),
 (120, 'Maryland', 'Food & Fitness', '30'),
 (121, 'Massachusetts', 'Food & Fitness', '51'),
 (122, 'Michigan', 'Food & Fitness', '21'),
 (123, 'Minnesota', 'Food & Fitness', '39'),
 (

# Inserting values into category_by_state given list of tuples
Here, we call our list of tuples with the needed data to insert, our database, and the iqry statement below in the sql_insert_many function to populate the category_by_state table in our database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 4 total fields in the category table of our database, we have 4 question marks in this iqry
iqry = 'INSERT INTO category_by_state VALUES(?,?,?,?)'
#Insert values into the category table of our database
sql_insert_many(db,iqry,LoT)

# Using the execute_sql function to check that our insert worked

We use this cell to make sure that the value insert into the category_by_state table of our database that we completed above actually worked. We use our execute_sql function to call the data from the category_by_state and output a pandas data frame. We use this towards our final database as a double check to make sure everything works correctly.

In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM category_by_state')
cols='CategoryID,State,Category,Ranking'
category_by_state=pd.DataFrame(data,columns=cols.split(','))
category_by_state

,CategoryID,State,Category,Ranking
0,101,Alabama,Food & Fitness,3
1,102,Alaska,Food & Fitness,46
2,103,Arizona,Food & Fitness,25
3,104,Arkansas,Food & Fitness,6
4,105,California,Food & Fitness,26
...,...,...,...,...
145,246,Virginia,Obesity & Overweight Prevalence,24
146,247,Washington,Obesity & Overweight Prevalence,37
147,248,West Virginia,Obesity & Overweight Prevalence,1
148,249,Wisconsin,Obesity & Overweight Prevalence,29


# Inserting values into the median income and unemployment tables of our database





# Obtaining a list of tuples from our median income dataframe in this notebook
Here, we call all the columns in our state_medincome dataframe created in this notebook inside the zip function to create an object that has individual tuples with the element of each tuple corresponding to a given value below one of the columns of our state_medincome dataframe (e.g. the element at index 0 of each tuple corresponds to the StateID column, the element at index 1 of each tuple corresponds to the State column...). We then call this zip function with its arguments inside the list function to obtain the needed list of tuples from our state_medincome data in this notebook. This list of tuples will be used to populate the state_medincome table in our database.

In [ ]:
#Obtain list of tuples given our tidy median income table (state_medincome) above
LoT=list(zip(state_medincome['StateID'],state_medincome['State'],state_medincome['HouseholdIncome']))
LoT

[(101, 'Alabama', 52035),
 (102, 'Alaska', 77790),
 (103, 'Arizona', 61529),
 (104, 'Arkansas', 49475),
 (105, 'California', 78672),
 (106, 'Colorado', 75231),
 (107, 'Connecticut', 79855),
 (108, 'Delaware', 69110),
 (109, 'Florida', 57703),
 (110, 'Georgia', 61224),
 (111, 'Hawaii', 83173),
 (112, 'Idaho', 58915),
 (113, 'Illinois', 68428),
 (114, 'Indiana', 58235),
 (115, 'Iowa', 61836),
 (116, 'Kansas', 61091),
 (117, 'Kentucky', 52238),
 (118, 'Louisiana', 50800),
 (119, 'Maine', 59489),
 (120, 'Maryland', 87063),
 (121, 'Massachusetts', 84385),
 (122, 'Michigan', 59234),
 (123, 'Minnesota', 73382),
 (124, 'Mississippi', 46511),
 (125, 'Missouri', 57290),
 (126, 'Montana', 56539),
 (127, 'Nebraska', 63015),
 (128, 'Nevada', 62043),
 (129, 'New Hampshire', 77923),
 (130, 'New Jersey', 85245),
 (131, 'New Mexico', 51243),
 (132, 'New York', 71117),
 (133, 'North Carolina', 56642),
 (134, 'North Dakota', 65315),
 (135, 'Ohio', 58116),
 (136, 'Oklahoma', 53840),
 (137, 'Oregon', 65667

# Inserting values into state_medincome given list of tuples

Here we take the list of tuples that we created above and insert values using the sql_insert_many function into the state_medincome table of our final database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 3 total fields in the median income table of our database, we have 3 question marks in this iqry
iqry = 'INSERT INTO state_medincome VALUES(?,?,?)'
#Insert values into the median income table of our database
sql_insert_many(db,iqry,LoT)

# Using the execute_sql function to check that our insert worked
Here we call our execute_sql function with our database and another argument to access all the columns of the state_medincome table in our database. From there, we create a dataframe from this query execution to determine if our insert worked and it appears that the insert did work.

In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM state_medincome')
cols='StateID,State,HouseholdIncome'
state_medincome =pd.DataFrame(data,columns=cols.split(','))
state_medincome

,StateID,State,HouseholdIncome
0,101,Alabama,52035
1,102,Alaska,77790
2,103,Arizona,61529
3,104,Arkansas,49475
4,105,California,78672
5,106,Colorado,75231
6,107,Connecticut,79855
7,108,Delaware,69110
8,109,Florida,57703
9,110,Georgia,61224


# Obtaining list of tuples from our state unemployment table in this notebook

The goal of this cell is to take the data from the state_unemp data frame  and turn them into a list of tuples where all the columns of our state_unemp data in this notebook are called in the zip function (creates an object of tuples that "zip" together multiple lists/iterables) and the zip function with its arguments is called in the list function. This is important, because the next step is to be able to take this LoT and put the data from this list of tuples in the state_unemp table of our database.

In [ ]:
#Obtain list of tuples given our tidy state unemployment table (state_unemp) above
LoT=list(zip(state_unemp['soceconID'],state_unemp['State'],state_unemp['Unemp_Year'],state_unemp['Unemployment_rate']))
LoT

[(101, 'Alabama', 2022, 2.6),
 (102, 'Alaska', 2022, 4.5),
 (103, 'Arizona', 2022, 3.3),
 (104, 'Arkansas', 2022, 3.3),
 (105, 'California', 2022, 3.9),
 (106, 'Colorado', 2022, 3.3),
 (107, 'Connecticut', 2022, 3.7),
 (108, 'Delaware', 2022, 4.4),
 (109, 'Florida', 2022, 2.7),
 (110, 'Georgia', 2022, 2.8),
 (111, 'Hawaii', 2022, 4.1),
 (112, 'Idaho', 2022, 2.6),
 (113, 'Illinois', 2022, 4.4),
 (114, 'Indiana', 2022, 2.6),
 (115, 'Iowa', 2022, 2.5),
 (116, 'Kansas', 2022, 2.4),
 (117, 'Kentucky', 2022, 3.7),
 (118, 'Louisiana', 2022, 3.6),
 (119, 'Maine', 2022, 2.8),
 (120, 'Maryland', 2022, 3.9),
 (121, 'Massachusetts', 2022, 3.5),
 (122, 'Michigan', 2022, 4.2),
 (123, 'Minnesota', 2022, 1.8),
 (124, 'Mississippi', 2022, 3.6),
 (125, 'Missouri', 2022, 2.5),
 (126, 'Montana', 2022, 2.7),
 (127, 'Nebraska', 2022, 2.0),
 (128, 'Nevada', 2022, 4.4),
 (129, 'New Hampshire', 2022, 2.0),
 (130, 'New Jersey', 2022, 3.7),
 (131, 'New Mexico', 2022, 4.5),
 (132, 'New York', 2022, 4.4),
 (133, '

# Inserting values into state_unemp given list of tuples
Here, we call our list of tuples with the needed data to insert, our database, and the iqry statement below in the sql_insert_many function to populate the state_unemp table in our database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 4 total fields in the state unemployment table of our database, we have 4 question marks in this iqry
iqry = 'INSERT INTO state_unemp VALUES(?,?,?,?)'
#Insert values into the state unemployment table of our database
sql_insert_many(db,iqry,LoT)

# Using execute_sql function to check that the insert worked

Here we are running the execute_sql function that we created above and using it to check that the data we just inserted into our state_unemp table in the database actually works. This is important because we know that based on this, the table in our database in correct.

In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM state_unemp')
cols='StateID,State,Unemp_Year,Unemployment_Rate'
state_unemp =pd.DataFrame(data,columns=cols.split(','))
state_unemp

,StateID,State,Unemp_Year,Unemployment_Rate
0,101,Alabama,2022,2.6
1,102,Alaska,2022,4.5
2,103,Arizona,2022,3.3
3,104,Arkansas,2022,3.3
4,105,California,2022,3.9
...,...,...,...,...
95,196,Virginia,2021,3.8
96,197,Washington,2021,5.2
97,198,West Virginia,2021,5.1
98,199,Wisconsin,2021,3.8


# Inserting values into our fastfood tables of our database

# Obtaining a list of tuples from our state_fast_food data in this notebook
Here, we call all the columns in our state_fast_food dataframe created in this notebook inside the zip function to create an object that has individual tuples with the element of each tuple corresponding to a given value below one of the columns of our state_fast_food dataframe (e.g. the element at index 0 of each tuple corresponds to the StateID column, the element at index 1 of each tuple corresponds to the State column...). We then call this zip function with its arguments inside the list function to obtain the needed list of tuples from our state_fast_food data in this notebook. This list of tuples will be used to populate the state_fast_food table in our database.

In [ ]:
#Obtain list of tuples given our tidy state_fast_food table above
LoT3=list(zip(state_fast_food['StateID'],state_fast_food['State'],state_fast_food['All fast food restaurants'],state_fast_food['Full-service restaurants'],state_fast_food['mostPopular']))
LoT3

[(101, 'Alabama', '81.7', '57.6', 'Chick-Fil-A'),
 (102, 'Alaska', '61.9', '78.7', "McDonald's"),
 (103, 'Arizona', '67.9', '59.2', 'In-N-Out'),
 (104, 'Arkansas', '69.9', '67.6', 'Chick-Fil-A'),
 (105, 'California', '82.3', '80.4', 'In-N-Out'),
 (106, 'Colorado', '75.7', '88.2', 'Chick-Fil-A'),
 (107, 'Connecticut', '76.1', '97.1', 'Panda Express'),
 (108, 'Delaware', '78.3', '85.5', 'Chick-Fil-A'),
 (109, 'Florida', '65.5', '78.9', 'Chick-Fil-A'),
 (110, 'Georgia', '82.5', '69.4', 'Chick-Fil-A'),
 (111, 'Hawaii', '97.5', '89.9', 'Panda Express'),
 (112, 'Idaho', '65.5', '74.4', 'Chick-Fil-A'),
 (113, 'Illinois', '82.5', '77.7', 'White Castle'),
 (114, 'Indiana', '73.7', '69.7', "McDonald's"),
 (115, 'Iowa', '67.7', '79.1', "Wendy's"),
 (116, 'Kansas', '74.7', '75.4', 'Jack In the Box'),
 (117, 'Kentucky', '74.2', '63.2', 'Chick-Fil-A'),
 (118, 'Louisiana', '76.2', '65.0', 'Chick-Fil-A'),
 (119, 'Maine', '74.9', '110.3', "McDonald's"),
 (120, 'Maryland', '89.7', '59.0', 'Chick-Fil-A')

# Inserting values into state_fast_food of our database given list of tuples

Here we are using our sql_insert_many function to insert all of the desired values from the LoT3 into the state_fast_food table of our final database. This is important because we needed values in our table and now that we have the values along with the columns, we now have a full table in our database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 5 total fields in the state_fast_food table of our database, we have 5 question marks in this iqry
iqry = 'INSERT INTO state_fast_food VALUES(?,?,?,?,?)'
#Insert values into the fastfood table of our database
sql_insert_many(db,iqry,LoT3)

# Using the execute_sql function to check that our insert worked

Here we call our execute_sql function with our database and another argument to access all the columns of the state_fast_food table in our database. From there, we create a dataframe from this query execution to determine if our insert worked and it appears that the insert did work.

In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM state_fast_food')
cols='StateID,State,All fast food restaurants,Full-service restaurants,mostPopular'
state_ff1 =pd.DataFrame(data,columns=cols.split(','))
state_ff1

,StateID,State,All fast food restaurants,Full-service restaurants,mostPopular
0,101,Alabama,81.7,57.6,Chick-Fil-A
1,102,Alaska,61.9,78.7,McDonald's
2,103,Arizona,67.9,59.2,In-N-Out
3,104,Arkansas,69.9,67.6,Chick-Fil-A
4,105,California,82.3,80.4,In-N-Out
5,106,Colorado,75.7,88.2,Chick-Fil-A
6,107,Connecticut,76.1,97.1,Panda Express
7,108,Delaware,78.3,85.5,Chick-Fil-A
8,109,Florida,65.5,78.9,Chick-Fil-A
9,110,Georgia,82.5,69.4,Chick-Fil-A


# Obtaining a list of tuples from our state_numfastfood data in this notebook

The goal of this cell is to take the data from the state_numfastfood data frame  and turn them into a list of tuples where all the columns of our state_numfastfood data in this notebook are called in the zip function (creates an object of tuples that "zip" together multiple lists/iterables) and the zip function with its arguments is called in the list function. This is important, because the next step is to be able to take this LoT and put it in the state_numfastfood table of our data base.

In [ ]:
LoT3=list(zip(state_numfastfood['FastFoodID'],state_numfastfood['State'],state_numfastfood['Fast Food Resturant'],state_numfastfood['Number Per 100 people']))
LoT3

[(101, 'Alabama', 'Subway', '8.2'),
 (102, 'Alaska', 'Subway', '7.2'),
 (103, 'Arizona', 'Subway', '5.6'),
 (104, 'Arkansas', 'Subway', '8.4'),
 (105, 'California', 'Subway', '5.7'),
 (106, 'Colorado', 'Subway', '6.7'),
 (107, 'Connecticut', 'Subway', '8.1'),
 (108, 'Delaware', 'Subway', '2.5'),
 (109, 'Florida', 'Subway', '6.0'),
 (110, 'Georgia', 'Subway', '7.1'),
 (111, 'Hawaii', 'Subway', '7.0'),
 (112, 'Idaho', 'Subway', '6.9'),
 (113, 'Illinois', 'Subway', '7.8'),
 (114, 'Indiana', 'Subway', '8.9'),
 (115, 'Iowa', 'Subway', '8.9'),
 (116, 'Kansas', 'Subway', '8.0'),
 (117, 'Kentucky', 'Subway', '8.5'),
 (118, 'Louisiana', 'Subway', '9.3'),
 (119, 'Maine', 'Subway', '8.2'),
 (120, 'Maryland', 'Subway', '6.8'),
 (121, 'Massachusetts', 'Subway', '4.7'),
 (122, 'Michigan', 'Subway', '8.0'),
 (123, 'Minnesota', 'Subway', '7.4'),
 (124, 'Mississippi', 'Subway', '9.6'),
 (125, 'Missouri', 'Subway', '8.0'),
 (126, 'Montana', 'Subway', '7.0'),
 (127, 'Nebraska', 'Subway', '9.6'),
 (128, '

# Inserting values into state_numfastfood given list of tuples
Here, we call our list of tuples with the needed data to insert, our database, and the iqry statement below in the sql_insert_many function to populate the state_numfastfood table in our database.

In [ ]:
db = 'US_State_Food.db'
#Since there are 4 total fields in the state_numfastfood table of our database, we have 4 question marks in this iqry
iqry = 'INSERT INTO state_numfastfood VALUES(?,?,?,?)'
#Insert values into the fastfood table of our database
sql_insert_many(db,iqry,LoT3)

# Using execute_sql function to check that our insert worked

Here we call our execute_sql function with our database and another argument to access all the columns of the state_numfastfood table in our database. From there, we create a dataframe from this query execution to determine if our insert worked and it appears that the insert did work.

In [ ]:
#Check that the insert worked
data=execute_sql(db,'SELECT * FROM state_numfastfood')
cols='FastFoodID,State,Fast_Food_Restaurant,Number_per_100_People'
state_ff2 =pd.DataFrame(data,columns=cols.split(','))
state_ff2

,FastFoodID,State,Fast_Food_Restaurant,Number_per_100_People
0,101,Alabama,Subway,8.2
1,102,Alaska,Subway,7.2
2,103,Arizona,Subway,5.6
3,104,Arkansas,Subway,8.4
4,105,California,Subway,5.7
...,...,...,...,...
495,596,Virginia,KFC,1.4
496,597,Washington,KFC,1.0
497,598,West Virginia,KFC,2.1
498,599,Wisconsin,KFC,1.1


# Challenges faced when we tried to store data in our database

There were a couple of problems that we had to and will have to navigate through with storing data in our database:
* When we initially put the table about the number of fast food restaurants per 100,000 people for each state in our notebook via webscraping (state_ff_df), multiple fast food restaurant names (values) were spanned over multiple columns in the table (Wendy’s, McDonald’s, Starbucks…were column names).
* The state_ff_df table we loaded in didn’t follow proper database design standards (tables aren’t supposed to have values in the columns), so we had to create a new table where we put all those fast food restaurants below a column called Fast Food Restaurant and create another column called Number per 100 people that had values for the number of each restaurant per 100,000 people (did this when we used the melt function to create the state_ff1 table above). That way, all of our fast food restaurant data for each state is now tidy and we can put subsets of data from that table into our database.
* Like the number of fast food restaurants per 100,000 people in each state that had fast food restaurant names as column names, two other tables we first loaded into this data acquisition file didn’t follow proper database standards. Those tables were our unemployment data for each state with year values (rate21,rate22) spanned over multiple columns (the table named employment at the top of our notebook) and our table for the obesity scores/categories across each state at the very top of this data acquisition notebook with the different categories (Health Consequences, Food & Fitness...) spanned over multiple columns (the table named obesity_df at the very top of our notebook).
* We converted these two tables to be tidy with the melt function above as well so they would follow proper database design standards and then we extracted subsets of data from these tables to create our database (when created the category_by_state and state_unemp tables in this notebook before creating our database).
* Finally, not all the tables in our database had the same number of rows, so we had to use different primary keys across our different tables (e.g. we had to use CategoryID in our category_by_state table, unempID in our state_unemp table, FastFoodID in our state_numfastfood table…). This will lead to some challenges when we have to combine tables for analysis in our next notebook.


# Summarizing our database design
To end off this data acquisition notebook, we want to show a summary of all the tables that are now stored in our database with columns and populated values. We summarize all our tables and description of the variables of all our tables below. Our database follows good design standards since none of the tables in it have values as column names, multiple values in different cells, or redundant columns.

# Table 1: obesity_by_state

This table contains data pertaining to the total obesity scores and obesity ranking of each state (50 rows).

Table Field Name|Field Description
-|-
StateID|The primary key for the states in the table
State|A United States state
Overall_Rank|Numerical ranking based on total obesity score
Total_Score|The total obesity score of each state


# Table 2: category_by_state

This table hows the rank for different health categories across each state (150 rows). The different columns for this table in our database are described in the table below.

Table Field Name|Field Description
-|-
CategoryID|Primary key column that uniquely identifies each row for this category table
State|A US State
Category|The category in a given State (Food & Fitness, Health Consequences, Obesity & Overweight Prevalence)
Ranking|The ranking for a given category across each State. A rank of 1 indicates that the state is worst in that category while a rank closer to 50 indicates that a state is above average in that category.

# Table 3: state_numfastfood

This table contains data about which and the amount of fast food resturants per 100,000 people in each state (500 rows).

Table Field Name|Field Description
-|-
FastFoodID|The primary key for the resturants in the table
State|A United States state
Fast_Food_Resturant|Name of a fast food resturant in each respective state
Number_Per_100_People|Number of each resturant in each state per 100,000 people

# Table 4: state_fast_food

This table shows the data for all fast food restaurants, full service restaurants, and most popular fast food restaurants in each state       (50 rows). The different columns for this table in our database are described in the table below.

Table Field Name|Field Description
-|-
StateID|Primary key column that uniquely identifies each row for this fast food table
State|A US State
All_fast_food_restaurants|The total number of fast food restaurants per 100,000 people in each state
Full_service_restaurants|The total number of full service restaurants per 100,000 people in each state
mostPopular|The most popular fast food restaurant in each state

#Table 5: state_medincome

This table contains data pertaining to each individual states' median household income (50 rows).

Table Field Name|Field Description
-|-
StateID|The primary key for the states in the table
State|A United States state
HouseholdIncome|The median household income for each state

# Table 6: state_unemp

This table shows the data for the unemployment rates in 2021 and 2022 for the US states (100 rows).

Table Field Name|Field Description
-|-
unempID|Primary key column that uniquely identifies each row for this unemployment table
State|A US State
Unemp_Year|The year for a given unemployment rate in this table
Unemployment_rate|The unemployment rate across multiple years for each state